# Worksheet 08

Name: Yuchen Cao  
UID: U51424608

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [514]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.554279319419288, 4.556730795225612, 4.331551491997016, 4.846598065856022, 2.839904959828095, 3.4804060411172797, 4.768267746381696, 3.801596928895064, 4.395750821792968, 5.112361821418937]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [515]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.412089809412819, 10.227046074455503, 7.289399624853598, 8.346030807428695, 7.911325043348029, 7.984911470825929, 9.137429834851885, 9.002757781712486, 8.069998456464125, 7.15173036024846]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [516]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.15173036024846, 5.112361821418937, 4.395750821792968, 3.801596928895064, 4.768267746381696, 8.069998456464125, 3.4804060411172797, 9.002757781712486, 9.137429834851885, 7.984911470825929]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean of the first normal distribution: 5  
The standard deviation of the first normal distribution: 1  
The number of data in the first normal distribution: 10  
The mean of the second normal distribution: 8  
The standard deviation of the second normal distribution: 1  
The number of data in the second normal distribution: 10  
The possibility of choosing data from each distribution is both 0.5.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [517]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.112361821418937, 4.395750821792968, 3.801596928895064, 4.768267746381696, 3.4804060411172797]
[7.15173036024846, 8.069998456464125, 9.002757781712486, 9.137429834851885, 7.984911470825929]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.311676671921189,  mean_2 = 8.269365580820576
var_1 = 0.36156655729252424,  var_2 = 0.5322339093372601


The result of code is:  
P(S_1) = 0.5,  P(S_2) = 0.5  
mean_1 = 4.311676671921189,  mean_2 = 8.269365580820576  
var_1 = 0.36156655729252424,  var_2 = 0.5322339093372601  
I think all probability and mean are close to the true values, standard deviation is a little bit different.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [518]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    #print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        #print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


7.15173036024846
Probability of coming from S_1 = 5.342165780088853e-13
Probability of coming from S_2 = 0.9999999999994658

5.112361821418937
Probability of coming from S_1 = 0.9999998193406432
Probability of coming from S_2 = 1.8065935682281966e-07

4.395750821792968
Probability of coming from S_1 = 0.9999999999978042
Probability of coming from S_2 = 2.195814135540074e-12

3.801596928895064
Probability of coming from S_1 = 0.999999999999999
Probability of coming from S_2 = 9.180694212363733e-16

4.768267746381696
Probability of coming from S_1 = 0.9999999993945875
Probability of coming from S_2 = 6.054125393121626e-10

8.069998456464125
Probability of coming from S_1 = 5.449358202448965e-24
Probability of coming from S_2 = 1.0

3.4804060411172797
Probability of coming from S_1 = 1.0
Probability of coming from S_2 = 2.5082007101808756e-17

9.002757781712486
Probability of coming from S_1 = 1.0645506885625439e-36
Probability of coming from S_2 = 1.0

9.137429834851885
Probability of co

It seems close to the truth value.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [519]:
prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum((x - mean[0])**2 * prob for x, prob in zip(data, prob_s0_x)) / sum(prob_s0_x), sum((x - mean[1])**2 * prob for x, prob in zip(data, prob_s1_x)) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4999999818733568,  P(S_2) = 0.5000000181266432
mean_1 = 4.311676642935917,  mean_2 = 8.269365466326622
var_1 = 0.361566547212091,  var_2 = 0.532234251647195


The values are:  
P(S_1) = 0.4999999818733568,  P(S_2) = 0.5000000181266432  
mean_1 = 4.311676642935917,  mean_2 = 8.269365466326622  
var_1 = 0.361566547212091,  var_2 = 0.532234251647195  
I think it seems not a big difference between this and K means.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [520]:
change = 0.000001
max_iter = 10

prev_mean = np.copy(mean)
prev_var = np.copy(var)
prev_prob_s = np.copy(prob_s)

for iteration in range(max_iter):
    prob_s0_x = [] # P(S_0 | X_i)
    prob_s1_x = [] # P(S_1 | X_i)
    prob_x = [] # P(X_i)

    k = 2

    for p in data:
        #print("point = ", p)
        pdf_i = []

        for j in range(k):
            # P(X_i | S_j)
            pdf_i.append(norm.pdf(p, mean[j], var[j]))
            #print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
            # P(S_j) already computed
            prob_s[j]

        # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
        prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]
        # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
        prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
        prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

    prob_s = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
    mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
    var = [ sum((x - mean[0])**2 * prob for x, prob in zip(data, prob_s0_x)) / sum(prob_s0_x) + 1e-6, sum((x - mean[1])**2 * prob for x, prob in zip(data, prob_s1_x)) / sum(prob_s1_x) + 1e-6 ]

    
    mean_converged = np.all(np.abs(mean - prev_mean) < change)
    var_converged = np.all(np.abs(var - prev_var) < change)
    prob_s_converged = np.all(np.abs(prob_s - prev_prob_s) < change)

    if mean_converged and var_converged and prob_s_converged:
        print(f"EM algorithm converged after {iteration + 1} iterations.")
        break

    prev_mean = np.copy(mean)
    prev_var = np.copy(var)
    prev_prob_s = np.copy(prob_s)

print("Final cluster probabilities: P(S_1) = {}, P(S_2) = {}".format(prob_s[0], prob_s[1]))
print("Final means: mean_1 = {}, mean_2 = {}".format(mean[0], mean[1]))
print("Final variances: var_1 = {}, var_2 = {}".format(var[0], var[1]))


EM algorithm converged after 2 iterations.
Final cluster probabilities: P(S_1) = 0.4999999818719584, P(S_2) = 0.5000000181280416
Final means: mean_1 = 4.311676642933681, mean_2 = 8.26936546631779
Final variances: var_1 = 0.3615675472113149, var_2 = 0.5322352516736136


The result is:  
EM algorithm converged after 2 iterations.  
Final cluster probabilities: P(S_1) = 0.4999999818719584, P(S_2) = 0.5000000181280416  
Final means: mean_1 = 4.311676642933681, mean_2 = 8.26936546631779  
Final variances: var_1 = 0.3615675472113149, var_2 = 0.5322352516736136  
It seems getting better by iterations. All three values are getting closer to the truth values.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [525]:
cluster_0 = []
cluster_1 = []
for i in range(len(prob_s0_x)):
    if prob_s0_x[i] > prob_s1_x[i] :
        cluster_0.append(data[i])
    else:
        cluster_1.append(data[i])
print("cluster 0 is:")
print(cluster_0)
print("cluster 1 is:")
print(cluster_1)

cluster 0 is:
[5.112361821418937, 4.395750821792968, 3.801596928895064, 4.768267746381696, 3.4804060411172797]
cluster 1 is:
[7.15173036024846, 8.069998456464125, 9.002757781712486, 9.137429834851885, 7.984911470825929]
